# Get SRTM data through a BMI

This notebook describes how to download Shuttle Radar Topography Mission (SRTM) elevation data
using a [Basic Model Interface](https://bmi.readthedocs.io/) (BMI) through the `BmiTopography` class included in the `bmi-topography` package.

## Setup

To ensure all dependencies are met, set up a conda environment using the environment file found in the root directory of this repository:
```
conda env create --file=environment.yml
```

Then install the `bmi-topography` package:
```
make install
```

Import a pair of libraries for later use:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Fetch and load data

Import the `BmiTopography` class from the newly installed `bmi-topography` package:

In [ ]:
from bmi_topography import BmiTopography

Create an instance of this class.

In [ ]:
m = BmiTopography()

Calling `help` on the instance displays all the BMI methods that are available.

In [ ]:
help(m)

The first step in using a BMI is calling the `initialize` method.
This method requires a configuration file that provides initial values for the `Topography` library wrapped by the BMI.

A sample configuration file is provided in the current directory.

In [ ]:
!ls

In [ ]:
!cat config.yaml

Call `initialize` with the sample configuration file.

In [ ]:
m.initialize("config.yaml")

This step may take a moment, as the `Topography` library fetches and downloads the data from the internet.

Once the step completes, check the contents of your cache directory:

In [ ]:
!ls ~/.bmi_topography

## Access data through the BMI

Now that we've fetched the data, let's access it through the BMI.
This will take a few steps.
It may seem cumbersome at first, but there's payoff at the end.

Start by displying the name of the one variable exposed through the BMI.

In [ ]:
m.get_output_var_names()

The (long) name for the variable representing elevation is an instance of a [CSDMS Standard Name](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names).
Standard Names are intended to be unambiguous; the tradeoff is that they tend to be long.

Find the data type of the elevation data.

In [ ]:
dtype = m.get_var_type("land_surface__elevation")
dtype

Within the BMI, functions that describe the grids that variables are defined on take an index instead of a variable name.

Get the grid index for the elevation variable.

In [ ]:
grid = m.get_var_grid("land_surface__elevation")
grid

Then find the total size of the elevation data.

In [ ]:
size = m.get_grid_size(grid)
size

Next, get the elevation values.

Two notes, however:

* As a rule, memory should not be allocated within a BMI. This leads to the un-Pythonic way that we get the elevation data--first creating an empty array, then passing it to a BMI function to receive values.
* BMI arrays are flattened. This obviates array ordering issues between languages, but it does make >1D data harder to work with.

Allocate an array for the elevation data.

In [ ]:
elevation = np.ndarray(size, dtype)
elevation

Get the elevation data.

In [ ]:
m.get_value("land_surface__elevation", elevation)

Note that the elevation array is one-dimensional.

In [ ]:
elevation.shape

### Reshape data

Like all BMI arrays, the elevations returned from the BMI `get_value` function are flattened.
Let's restore their original dimensionality.

First, determine the dimensionality of the elevation variable.

In [ ]:
rank = m.get_grid_rank(grid)
rank

Get the dimensions of the elevation data, first creating an array to store their values.

In [ ]:
shape = np.ndarray(rank, dtype=int)
shape

In [ ]:
m.get_grid_shape(grid, shape)

Reshape the elevation data, creating a new array.

In [ ]:
elevation2D = elevation.reshape(shape)

In [ ]:
elevation2D.shape

## Visualize

Let's visualize the elevation data as an image.

In [ ]:
plt.imshow(elevation2D)

## Conclusion

Last, call the BMI `finalize` function.

In [ ]:
m.finalize()

This demonstration of the BMI took a lot of code to reproduce a simple result.
So why would anyone want to use the BMI?
The key is that, in this demonstration, only the functions belonging to the BMI were used to access the data.
No knowledge of the calling syntax of the underlying `Topography` class was used.

The lesson is: once you've seen one BMI, you've seen them all!